In [1]:
import pandas as pd
from pathlib import Path
import os
import numpy as np

In [2]:
df = pd.read_excel('../data/12.xlsx', na_values='None')
df.head()

,name,keywords,top_kb_index,second_kb_index,third_kb_index,emoji
0,Ноутбук во временное пользование,Ноутбук,NaN,NaN,NaN,NaN
1,Доступ к сетевым папкам,"Сетевые папки, сетевая папка, сетевой диск",NaN,NaN,NaN,NaN
2,"Поддержка рабочих мест, настройка удаленки","Компьютер, ПК, сканер, принтер, МФУ, телефон, ...",NaN,NaN,NaN,NaN
3,"ИС УОП, 1С:БГУ 2.0, 1С:ЗКГУ 3.0, 1С:ФПЗ","1С, БГУ, ФПЗ, УОП, ЗКГУ, ЗКБУ, Сервер, удаленн...",NaN,NaN,NaN,NaN
4,1С: Документооборот,"1С, Документооборот, ДО",NaN,NaN,NaN,NaN


In [3]:
data_path = Path(r'C:\Users\maxxx\VSprojects\bots\MIPT\backend\data')
top_kb_file = data_path / Path('kb_top_level.csv')
second_kb_file = data_path / Path('kb_second_level_oldmipt.csv')
third_kb_file = data_path / Path('kb_third_level_oldmipt.csv')

In [4]:
top_df = pd.read_csv(top_kb_file, sep=';', na_values='None')
second_df = pd.read_csv(second_kb_file, sep=';', na_values='None')
third_df = pd.read_csv(third_kb_file, sep=';', na_values='None')

In [5]:
# def f(row):
#     #print(row['name'])
#     print(row)

# df.apply(f);

In [6]:
for idx, row in df.iterrows():
    if row['name'] in top_df['name'].values:
        row['top_kb_index'] = top_df[top_df['name'] == row['name']]['top_kb_index'].values[0]
        row['emoji'] = top_df[top_df['name'] == row['name']]['emoji'].values[0]
        df.loc[idx] = row
    elif row['name'] in second_df['name'].values:
        row['top_kb_index'] = second_df[second_df['name'] == row['name']]['top_kb_index'].values[0]
        row['second_kb_index'] = second_df[second_df['name'] == row['name']]['second_kb_index'].values[0]
        row['emoji'] = second_df[second_df['name'] == row['name']]['emoji'].values[0]
        df.loc[idx] = row
    elif row['name'] in third_df['name'].values:
        row['top_kb_index'] = third_df[third_df['name'] == row['name']]['top_kb_index'].values[0]
        row['second_kb_index'] = third_df[third_df['name'] == row['name']]['second_kb_index'].values[0]
        row['third_kb_index'] = third_df[third_df['name'] == row['name']]['third_kb_index'].values[0]
        row['emoji'] = third_df[third_df['name'] == row['name']]['emoji'].values[0]
        df.loc[idx] = row
    else:
        # print(f'error string = {idx}; {row["name"]}')
        ...

df[df['emoji'] == None]

,name,keywords,top_kb_index,second_kb_index,third_kb_index,emoji


In [7]:
# df[df['emoji'] == None]
df.to_excel('../data/keywords.xlsx', index=False)

In [8]:
df = pd.read_excel('../data/keywords.xlsx')
df.isna().sum()

name                0
keywords            0
top_kb_index        0
second_kb_index     0
third_kb_index     38
emoji               0
dtype: int64

In [9]:
def proccess_kw(words: str):
    words = words.lower()
    words = words.replace(';', '')
    words = words.replace(',', '')
    words = words.replace('.', '')
    words = words.replace('-', ' ')

    words_arr = [word for word in words.split(' ') if len(word) > 1]

    return words_arr


In [10]:
# for el in df['keywords'].apply(proccess_kw):
#     print(el)
df['keywords'] = df['keywords'].apply(proccess_kw)

In [11]:
my_word = 'почта'

# df[pd.isin]
# df[df['keywords'].isin([my_word])]

In [13]:
import nltk
nltk.download('stopwords')
import re
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()
def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    print(f'doc  = {doc}')
    tokens = []
    for token in doc.split():
        print(f'token = {token}')
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token)
    # print(tokens)
    if len(tokens) > 0:
        return tokens
    return None

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maxxx\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
lemmatize('ошибку')

doc  = ошибку
token = ошибку


['ошибка']

In [113]:
request = ['мои документы']
res = lemmatize(request)
print(res)

TypeError: expected string or bytes-like object

In [44]:
from collections import defaultdict 

def get_kw2idx(df: pd.DataFrame) -> dict[set]:
    kw2idx = defaultdict(set)
    for idx, row in df.iterrows():
        for word in row['keywords']:
            kw2idx[word].add(idx)

        # print(row['keywords'])
        # print(lemmatize(' '.join(row['keywords'])))

        lemmatize_keywords = lemmatize(' '.join(row['keywords']))
        for word in lemmatize_keywords if lemmatize_keywords is not None else []:
            kw2idx[word].add(idx)
            ...

    return kw2idx

In [45]:
kw2idx = get_kw2idx(df)
kw2idx.__len__()

185

In [ ]:
my_word = 'документы'

In [121]:
for idx in kw2idx[my_word]:
    print(df.loc[idx, 'name'])
    # print(idx)

Корп. аккаунт: получить и восстановить
Изменить пароль от корп. сервисов
Корп. аккаунт @mipt.ru для сотрудников базовых кафедр, дистанционных работников
Получение почты @mipt.ru для подразделения и мероприятия 
Получение @phystech.edu студентами, аспирантами, выпускниками
Получение @phystech.edu сотрудниками
Получение @phystech.edu для подразделения и мероприятия
Восстановление пароля от @phystech.edu 


In [46]:
import requests 

In [118]:
URL = 'http://speller.yandex.net/services/spellservice.json/checkText'

response = requests.post(
    URL,
    data={
        'text': 'ошипка в текве',
        'lang': 'ru, en',
        'format': 'plain'
    }
)
response.json()

[{'code': 1,
  'pos': 0,
  'row': 0,
  'col': 0,
  'len': 6,
  'word': 'ошипка',
  's': ['ошибка']},
 {'code': 1,
  'pos': 9,
  'row': 0,
  'col': 9,
  'len': 5,
  'word': 'текве',
  's': ['тыкве', 'тексе', 'тейкове', 'тикве']}]

In [91]:
len(response.json())

5

In [88]:
response.status_code

200

In [73]:
response = requests.get(f'{URL}?text=какойв-то течст с ошипками точно&lang=ru&format=plain')

In [74]:
response.json()

[{'code': 1,
  'pos': 0,
  'row': 0,
  'col': 0,
  'len': 6,
  'word': 'какойв',
  's': ['какой']},
 {'code': 1,
  'pos': 10,
  'row': 0,
  'col': 10,
  'len': 5,
  'word': 'течст',
  's': ['текст', 'тест']},
 {'code': 1,
  'pos': 18,
  'row': 0,
  'col': 18,
  'len': 8,
  'word': 'ошипками',
  's': ['ошибками', 'ошипками']}]

In [105]:
request = 'мои документы почнму и 23 какой-то тектс'
res = lemmatize(request)
print(res)

['мой', 'документ', 'почнм', 'тектс']


In [108]:
res = lemmatize("fff что здесб такое?")
print(res)

None


In [ ]:
request.split()

['мои', 'документы']

In [98]:
print(res)

None
